In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive


In [3]:
%cd study/deep/styleGAN

/content/gdrive/My Drive/study/deep/styleGAN


In [4]:
!git clone https://github.com/pacifinapacific/StyleGAN_LatentEditor.git

fatal: destination path 'StyleGAN_LatentEditor' already exists and is not an empty directory.


In [5]:
%cd StyleGAN_LatentEditor

/content/gdrive/MyDrive/study/deep/styleGAN/StyleGAN_LatentEditor


In [6]:
!pip install tensorflow==1.15.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110.5 MB 36 kB/s 
     |████████████████████████████████| 3.8 MB 54.5 MB/s 
     |████████████████████████████████| 503 kB 64.7 MB/s 
     |████████████████████████████████| 50 kB 8.3 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=2990e92fe716ba90acde8cde01d68bdbb277a2a1620aa0f1f94a91e741829822
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting un

In [7]:



!python weight_convert.py

















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
sd only g_synthesis.blocks.8x8.conv0_up.intermediate.kernel torch.Size([1, 1, 3, 3])
sd only g_synthesis.blocks.16x16.conv0_up.intermediate.kernel torch.Size([1, 1, 3, 3])
sd only g_synthesis.blocks.32x32.conv0_up.intermediate.kernel torch.Size([1, 1, 3, 3])
sd only g_synthesis.blocks.64x64.conv0_up.intermediate.kernel torch.Size([1, 1, 3, 3])
sd only g_synthesis.blocks.128x128.conv0_up.intermediate.kernel torch.Size([1, 1, 3, 3])
sd only g_synthesis.blocks.256x256.conv0_up.intermediate.kernel torch.Size([1, 1, 3, 3])
sd only g_synthesis.blocks.512x512.conv0_up.intermediate.kernel torch.Size([1, 1, 3, 3])
sd only g_synthesis.blocks.1024x1024.conv0_up.intermediate.kernel torch.Size([1, 1, 3, 3])
sd only 1024x1024.blur.kernel torch.Size([1, 1, 3, 3])
sd only 1024x1024.conv1_down.downscale.blur.kernel torch.Size([1, 1, 2, 2])
sd only 512x512.blur.kernel torch.Size([1, 1, 3, 3])
sd on

In [8]:


!mkdir save_image
!mkdir save_image/encode1



mkdir: cannot create directory ‘save_image’: File exists
mkdir: cannot create directory ‘save_image/encode1’: File exists


copy and overwite this code on 'encode_image.py'



```
import numpy as np 
import matplotlib.pyplot as plt 
from stylegan_layers import  G_mapping,G_synthesis
from read_image import image_reader
import argparse
import torch
import torch.nn as nn
from collections import OrderedDict
import torch.nn.functional as F
from torchvision.utils import save_image
from perceptual_model import VGG16_for_Perceptual
import torch.optim as optim

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

dlatent1= torch.zeros((1,18,512),requires_grad=True,device=device)
dlatent2 = torch.zeros((1,18,512),requires_grad=True,device=device)

def main(dlatent, source_image, do_synthesis):
     parser = argparse.ArgumentParser(description='Find latent representation of reference images using perceptual loss')
     parser.add_argument('--batch_size', default=1, help='Batch size for generator and perceptual model', type=int)
     parser.add_argument('--resolution',default=1024,type=int)
     parser.add_argument('--src_im',default=source_image)
     parser.add_argument('--src_dir',default="source_image/")
     parser.add_argument('--weight_file',default="weight_files/pytorch/karras2019stylegan-ffhq-1024x1024.pt",type=str)
     parser.add_argument('--iteration',default=1000,type=int)



     args=parser.parse_args()

     g_all = nn.Sequential(OrderedDict([
    ('g_mapping', G_mapping()),
    #('truncation', Truncation(avg_latent)),
    ('g_synthesis', G_synthesis(resolution=args.resolution))    
    ]))




     g_all.load_state_dict(torch.load(args.weight_file, map_location=device))
     g_all.eval()
     g_all.to(device)


     g_mapping,g_synthesis=g_all[0],g_all[1]
     name=args.src_im.split(".")[0]
     img=image_reader(args.src_dir+args.src_im) #(1,3,1024,1024) -1~1
     img=img.to(device)

     MSE_Loss=nn.MSELoss(reduction="mean")

     img_p=img.clone() #Perceptual loss 用画像
     upsample2d=torch.nn.Upsample(scale_factor=256/args.resolution, mode='bilinear') #VGG入力のため(256,256)にリサイズ
     img_p=upsample2d(img_p)

     perceptual_net=VGG16_for_Perceptual(n_layers=[2,4,14,21]).to(device)
     
     optimizer=optim.Adam({dlatent},lr=0.01,betas=(0.9,0.999),eps=1e-8)

     print("Start")
     loss_list=[]
     
     for i in range(args.iteration):
          optimizer.zero_grad()
          synth_img=g_synthesis(dlatent)
          synth_img = (synth_img + 1.0) / 2.0
          mse_loss,perceptual_loss=caluclate_loss(synth_img,img,perceptual_net,img_p,MSE_Loss,upsample2d)
          loss=mse_loss+perceptual_loss
          loss.backward()

          optimizer.step()

          loss_np=loss.detach().cpu().numpy()
          loss_p=perceptual_loss.detach().cpu().numpy()
          loss_m=mse_loss.detach().cpu().numpy()

          loss_list.append(loss_np)
          if i%10==0:
               print("iter{}: loss -- {},  mse_loss --{},  percep_loss --{}".format(i,loss_np,loss_m,loss_p))
               save_image(synth_img.clamp(0,1),"save_image/encode1/{}_{}.png".format(i,source_image))
               #np.save("loss_list.npy",loss_list)
               np.save("latent_W/{}.npy".format(name),dlatent.detach().cpu().numpy())
     

     if do_synthesis:
       for i in range(10):
        interpolated_img =g_synthesis(dlatent1 * (i / 10) + dlatent2 * ((10 - i) / 10))
        interpolated_img = (interpolated_img + 1.0) / 2.0
        print("interpolated image {} was made".format(i))
        save_image(interpolated_img.clamp(0,1),"save_image/encode1/interpolated_{}.png".format(i))





        


          






          



def caluclate_loss(synth_img,img,perceptual_net,img_p,MSE_Loss,upsample2d):
     #calculate MSE Loss
     mse_loss=MSE_Loss(synth_img,img) # (lamda_mse/N)*||G(w)-I||^2

     #calculate Perceptual Loss
     real_0,real_1,real_2,real_3=perceptual_net(img_p)
     synth_p=upsample2d(synth_img) #(1,3,256,256)
     synth_0,synth_1,synth_2,synth_3=perceptual_net(synth_p)

     perceptual_loss=0
     perceptual_loss+=MSE_Loss(synth_0,real_0)
     perceptual_loss+=MSE_Loss(synth_1,real_1)
     perceptual_loss+=MSE_Loss(synth_2,real_2)
     perceptual_loss+=MSE_Loss(synth_3,real_3)

     return mse_loss,perceptual_loss




     







     

    

if __name__ == "__main__":
    main(dlatent1, "josh.png", False)
    main(dlatent2, "thanos.png", True)

```



In [9]:


!python encode_image.py  --iteration 1500



Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100% 528M/528M [00:03<00:00, 167MB/s]
Start
iter0: loss -- 10.125829696655273,  mse_loss --0.2494492530822754,  percep_loss --9.876380920410156
iter10: loss -- 5.2371368408203125,  mse_loss --0.2161749303340912,  percep_loss --5.020961761474609
iter20: loss -- 5.091344833374023,  mse_loss --0.17458996176719666,  percep_loss --4.916754722595215
iter30: loss -- 5.035895824432373,  mse_loss --0.1628619283437729,  percep_loss --4.8730340003967285
iter40: loss -- 5.020185470581055,  mse_loss --0.16089880466461182,  percep_loss --4.859286785125732
iter50: loss -- 5.007312774658203,  mse_loss --0.15403565764427185,  percep_loss --4.853277206420898
iter60: loss -- 4.957531929016113,  mse_loss --0.15423151850700378,  percep_loss --4.803300380706787
iter70: loss -- 4.9542717933654785,  mse_loss --0.15170128643512726,  percep_loss --4.802570343017578
iter80: loss -- 4.943

In [10]:
!python facial_exchange.py --src_im1  source_image/sample.png --src_im2  source_image/0.png  --iteration 200

Start
iter0:   loss0 --3.2650365829467773,  loss1 --5.065891741651285e-07
Traceback (most recent call last):
  File "facial_exchange.py", line 193, in <module>
    main()
  File "facial_exchange.py", line 87, in main
    loss_2.backward()
  File "/usr/local/lib/python3.7/dist-packages/torch/_tensor.py", line 363, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/__init__.py", line 175, in backward
    allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to run the backward pass
RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [1, 512]], which is output 0 of AsStridedBackward0, is at version 2; expected version 1 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).
